# AI Workshop - Lab 1-1: Data Cleaning and Processing

## Introduction

Welcome to the first lab of the Machine Learning workshop! In this session, we will explore the foundational steps of data cleaning and processing. We'll work with a dataset focusing on energy forecasting—an area where machine learning has high potential to support renewable energy transitions.

### Energy Market Forecasting Dataset

This dataset provides 4 years of data on electrical consumption, generation, pricing, and weather conditions for Spain. It includes:

- Energy consumption and generation: Data retrieved from ENTSOE, a public portal for Transmission Service Operator (TSO) data.
- Settlement prices: Data obtained from the Spanish TSO, Red Eléctrica España.
- Weather data: Sourced from the Open Weather API for the five largest cities in Spain.

This dataset is ideal for practicing preprocessing techniques because it integrates multiple data types (time series, categorical, and numerical). Forecasting energy trends from this data represents a real-world challenge with significant practical applications.

### Scikit-Learn

Scikit-Learn is a powerful Python library for machine learning. It includes tools for data preprocessing, model building, and evaluation. In this lab, we'll use Scikit-Learn alongside Pandas to:

- Inspect the dataset.
- Handle missing values.
- Prepare the data for machine learning models.

Let’s start by loading the dataset and inspecting it with Pandas to understand its structure and identify preprocessing needs!

In [ ]:
!pip install -Uq "ydata-profiling[notebook]"

In [ ]:
import pandas as pd

energy_dataset = pd.read_csv('data/energy_dataset.csv')
weather_features = pd.read_csv('data/weather_features.csv')
# Rename `dt_iso` column to `time` for consistency
weather_features = weather_features.rename(columns={'dt_iso': 'time'})

In [ ]:
# Look at some of the data
energy_dataset.head()

In [ ]:
weather_features.head()

Take a moment to familiarize yourself with the contents of the dataset. Try to answer some of the following questions. If you are new to Python, work in a group with someone who has familiarity with Python, or ask for help!

1. What cities are included in the weather data?
2. What are the columns in the energy dataset?
3. Do any of the datasets contain missing values?

In [ ]:
print(f'Cities in the weather data: {weather_features.city_name.unique()}')
print(f'Columns in the energy dataset: {energy_dataset.columns}')
print(f'Missing values in the energy dataset: {energy_dataset.isnull().sum()}')
print(f'Missing values in the weather data: {weather_features.isnull().sum()}')

We can also visualize many of the columns in the dataset to get a sense of the distribution of the data. In Google Colab this is as easy as printing out a column, but in other cases you may need to use a plotting library like Matplotlib or Seaborn. Let's start by looking at the distribution of the `generation solar` column in the energy dataset:

In [ ]:
import matplotlib.pyplot as plt

# Plot the distribution of the generation solar column against time (hour)

energy_dataset['time'] = pd.to_datetime(energy_dataset['time'], utc=True)
energy_dataset['hour'] = energy_dataset['time'].dt.hour
energy_dataset.plot(x='hour', y='generation solar', kind='scatter')
plt.title('Solar Generation by Hour')
plt.show()

### ydata-profiling

The `ydata-profiling` library is a powerful tool for generating detailed reports on the structure and contents of a dataset. It provides a quick way to understand the distribution of data, identify missing values, and detect potential issues. Let's use `ydata-profiling` to generate a report on the energy dataset:

_(Note: If the report doesn't seem to render properly in Google Colab, you can download the HTML file and open it in your browser.)_

In [ ]:
from ydata_profiling import ProfileReport

profile = ProfileReport(energy_dataset, title='Energy Dataset Profiling Report', explorative=True)
profile.to_file('energy_dataset_profile.html')
profile.to_notebook_iframe()

The profiling report provides a detailed overview of the dataset, including:

- The number of variables and observations.
- The presence of missing values.
- The distribution of numerical and categorical variables.
- The correlation between variables.

From this, you should be able to see that there are a few columns that always have a value of 0, indicating that they aren't being used. We will remove these later on.

In [ ]:
# Reload the initial data so we can do pre-processing
energy_dataset = pd.read_csv('data/energy_dataset.csv')
weather_features = pd.read_csv('data/weather_features.csv')
# Rename `dt_iso` column to `time` for consistency
weather_features = weather_features.rename(columns={'dt_iso': 'time'})

## Data Pre-Processing

Now that we have a better understanding of the dataset, we can start preparing the data for machine learning. This involves several steps, including:

1. Normalizing the data
2. Converting non-numeric data to numeric data
3. Handling missing values
4. Splitting the data into features and labels

Let's start with normalizing the data. As you have seen, our dataset contains a number of numerical columns that operate on different ranges. Depending on the machine learning model, this can pose a problem! Simply put, if you had no information about the data, which of the following would you think was more important:

- A column that ranges from 0 to 1
- A column that ranges from 0 to 1000?

By normalizing the data we can ensure that all of the columns are treated equally by the model. We can normalize the data by subtracting the mean and dividing by the standard deviation. This will ensure that the data has a mean of 0 and a standard deviation of 1. Let's normalize the data now:

In [ ]:
for column in energy_dataset.columns:
    if column != 'time' and energy_dataset[column].dtype != 'object':
        energy_dataset[column] = (energy_dataset[column] - energy_dataset[column].mean()) / energy_dataset[column].std()

for column in weather_features.columns:
    if column != 'time' and weather_features[column].dtype != 'object':
        weather_features[column] = (weather_features[column] - weather_features[column].mean()) / weather_features[column].std()

In [ ]:
# Look at the normalized data
energy_dataset.head()

There is one other thing we need to do before we can move forward. The model we will build requires that all of the data be numeric, but we have a few columns with text in them. We will convert the `city_name` and `weather_main` columns to numeric values using one-hot encoding. This will create a new column for each unique value in the original column, and fill it with `1` if the value is present and `0` if it is not. Let's do this now:

In [ ]:
# One-hot encode the city_name and weather_main columns
weather_features = pd.get_dummies(weather_features, columns=['city_name', 'weather_main'])

# Drop the weather_description column and the weather_icon column
weather_features = weather_features.drop(columns=['weather_description', 'weather_icon'])

# Look at the modified data
weather_features.head()

### Handling Missing Values

One of the most common problems in real-world datasets is missing values. Missing values can cause problems when building machine learning models, so it is important to handle them properly. There are several ways to handle missing values, including:

1. Removing rows or columns with missing values
2. "Imputing" missing values by filling them in with a best guess
3. Using a model that treats missing values as a separate category

We are going to look at each of these methods in turn, and see how they affect the performance of a machine learning model.

### Removing Rows with Missing Values

The simplest way to handle missing values is to remove any rows that contain missing values. This is a quick and easy way to handle missing values, but it can also lead to a loss of information. Pandas makes it easy to remove rows with missing values using the `dropna()` method. Let's see how this affects the energy demand dataset:

In [ ]:
# Remove rows with missing values
energy_dataset_dropped = energy_dataset.dropna()

# Look at the shape of the original and modified data
print(f'Original data shape: {energy_dataset.shape}')
print(f'Modified data shape: {energy_dataset_dropped.shape}')

...whoops! It looks like we have lost the significant majority of our dataset with this approach. If we inspect the data further, we can see which columns contain a significant number of missing values:

In [ ]:
# Report the number of missing values by column
missing_values = energy_dataset.isnull().sum()
missing_values = missing_values[missing_values > 0]
missing_values

As indicated earlier by ydata profiling, we can see that there are a number of columns which don't contain any information. We can remove these columns and then try removing rows with missing values again:

In [ ]:
missing_columns = [
    'generation fossil coal-derived gas',
    'generation fossil oil shale',
    'generation fossil peat',
    'generation geothermal',
    'generation hydro pumped storage aggregated',
    'generation marine',
    'generation wind offshore',
    'forecast wind offshore eday ahead'
]

In [ ]:
# Remove columns with a large number of missing values (and non-numeric columns)
energy_dataset_removed = energy_dataset.drop(columns=missing_columns)

# Remove rows with missing values
energy_dataset_removed = energy_dataset_removed.dropna()

# Look at the shape of the original and modified data
print(f'Original data shape: {energy_dataset.shape}')
print(f'Modified data shape: {energy_dataset_removed.shape}')

Okay, that loss of data is a bit easier to work with. We now know that every row in the dataset has complete information. There's an obvious tradeoff here, in that we have to decide whether it is better to have more data with missing values, or less data with complete information. However, there's also a slightly more subtle risk that this approach introduces. Can you think of what it might be? We'll come back to it later.

For now, let's build a very simple machine learning model to predict the current weather type based on the current energy generation. This is flipped from what you might have expected we would do! We will use a Random Forest model, which is a type of ensemble model that is easy to use and often performs well. We will use the `RandomForestClassifier` class from Scikit-Learn to build the model. Let's start by loading the data and splitting it into features and labels:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Merge energy demand and weather by time
df = pd.merge(energy_dataset_removed, weather_features, on='time', how='inner')

# Split the data into features and labels
X = df[['generation biomass', 'generation fossil brown coal/lignite',
        'generation fossil gas',
        'generation fossil hard coal', 'generation fossil oil',
        'generation hydro pumped storage consumption',
        'generation hydro run-of-river and poundage',
        'generation hydro water reservoir',
        'generation nuclear', 'generation other', 'generation other renewable',
        'generation solar', 'generation waste',
        'generation wind onshore']]

y = df[[column for column in df.columns if column.startswith('weather_main')]]

In [ ]:
# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Reset indices
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

In [ ]:
from sklearn.metrics import accuracy_score
from time import time

# Train a Random Forest model
model = RandomForestClassifier(n_jobs=-1)

# Fit the model to the training data
print('Fitting model...')
start_time = time()
model.fit(X_train, y_train)
print(f'Model fit in {time() - start_time:.0f} seconds')

# Evaluate the model on the testing data
print('Evaluating model...')
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

print(f'Model accuracy: {accuracy*100:.2f}%')

There is a bit of randomness here based on how we split the data, but hopefully you should see an accuracy of around 44%. This is not bad! Consider that we have 12 different weather types, so if we were just guessing randomly, we would expect an accuracy of around 8%. This is a significant improvement over random guessing.

Since we have 12 different types of weather, it's likely that the model is better at predicting some than it is for others. We can break down the accuracy by weather type to get a sense of how well the model is performing for each type:

In [ ]:
import numpy as np

print("Column                    | Count | Correct | Accuracy")
print("--------------------------|-------|---------|---------")
for i, column in enumerate(y_test.columns):
    count = y_test[column].sum()
    indices = y_test[y_test[column]].index
    predictions_for_column = predictions[indices]
    correct = predictions_for_column[:, i].sum()
    accuracy = correct / count if count > 0 else np.nan

    print(f'{column:<25} | {count:<5} | {correct:<7} | {accuracy:.2f}')

This is not good - we have just exposed a major flaw in our model. It seems like while there are numerous weather conditions that the model is able to predict, we have hardly any examples of weather outside of `clear`, `clouds`, and `rain`. This means that while our model is quite good at predicting `clear`, `clouds`, and `rain`, it is not very good at predicting any other weather type.

We are going to try to address this by grouping together all weather conditions aside from those three into a single category called `weather_main_other`:

In [ ]:
y_grouped = y[['weather_main_clear', 'weather_main_clouds', 'weather_main_rain']].copy()
y_grouped['weather_main_other'] = ~y_grouped.any(axis=1)

In [ ]:
y_grouped.sum(axis=0)

This is looking much more balanced - we now have a good number of examples of each weather type. Let's retrain our model and see how it performs:

In [ ]:
# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y_grouped, test_size=0.2)

# Reset indices
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# Train a Random Forest model
model = RandomForestClassifier()

# Fit the model to the training data
print('Fitting model...')
start_time = time()
model.fit(X_train, y_train)
print(f'Model fit in {time() - start_time:.0f} seconds')

# Evaluate the model on the testing data
print('Evaluating model...')
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

print(f'Model accuracy: {accuracy*100:.2f}')

In [ ]:
print("Column                    | Count | Correct | Accuracy")
print("--------------------------|-------|---------|---------")
for i, column in enumerate(y_test.columns):
    count = y_test[column].sum()
    indices = y_test[y_test[column]].index
    predictions_for_column = predictions[indices]
    correct = predictions_for_column[:, i].sum()
    accuracy = correct / count if count > 0 else np.nan

    print(f'{column:<25} | {count:<5} | {correct:<7} | {accuracy:.4f}')

Unfortunately, it looks like the model is still not performing well for the less common weather types. This is a common problem in machine learning, where the model is biased towards the most common classes. There are several ways to handle this, including:

1. Collecting more data for the less common classes
2. Using a different model that is better at handling imbalanced classes
3. Using techniques like oversampling or undersampling to balance the classes

We will look at some of these in more detail later in our workshop!

